In [1]:
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.factors import Latest
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.factors import CustomFactor, SimpleMovingAverage, AverageDollarVolume, Returns, RSI
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.filters import Q500US, Q1500US
from quantopian.pipeline.data.quandl import fred_usdontd156n as libor
from quantopian.pipeline.data.zacks import EarningsSurprises

import talib
import pandas as pd
import numpy as np
from time import time

import alphalens as al
import pyfolio as pf
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import linear_model, decomposition, ensemble, preprocessing, isotonic, metrics

In [2]:
bs = morningstar.balance_sheet
cfs = morningstar.cash_flow_statement
is_ = morningstar.income_statement
or_ = morningstar.operation_ratios
er = morningstar.earnings_report
v = morningstar.valuation
vr = morningstar.valuation_ratios


def make_factors():
    def Asset_Growth_3M():
        return Returns(inputs=[bs.total_assets], window_length=63)

    def Asset_To_Equity_Ratio():
        return bs.total_assets.latest / bs.common_stock_equity.latest

    def Capex_To_Cashflows():
        return (cfs.capital_expenditure.latest * 4.) / \
            (cfs.free_cash_flow.latest * 4.)
        
    def EBITDA_Yield():
        return (is_.ebitda.latest * 4.) / \
            USEquityPricing.close.latest        

    def EBIT_To_Assets():
        return (is_.ebit.latest * 4.) / \
            bs.total_assets.latest
        
    def Earnings_Quality():
        return morningstar.cash_flow_statement.operating_cash_flow.latest / \
               EarningsSurprises.eps_act.latest
        
    def Return_On_Total_Invest_Capital():
        return or_.roic.latest
    
    class Mean_Reversion_1M(CustomFactor):
        inputs = [Returns(window_length=21)]
        window_length = 252

        def compute(self, today, assets, out, monthly_rets):
            out[:] = (monthly_rets[-1] - np.nanmean(monthly_rets, axis=0)) / \
                np.nanstd(monthly_rets, axis=0)
                
    class MACD_Signal_10d(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 60

        def compute(self, today, assets, out, close):

            sig_lines = []

            for col in close.T:
                # get signal line only
                try:
                    _, signal_line, _ = talib.MACD(col, fastperiod=12,
                                                   slowperiod=26, signalperiod=10)
                    sig_lines.append(signal_line[-1])
                # if error calculating, return NaN
                except:
                    sig_lines.append(np.nan)
            out[:] = sig_lines 
            
    class Moneyflow_Volume_5d(CustomFactor):
        inputs = [USEquityPricing.close, USEquityPricing.volume]
        window_length = 5

        def compute(self, today, assets, out, close, volume):

            mfvs = []

            for col_c, col_v in zip(close.T, volume.T):

                # denominator
                denominator = np.dot(col_c, col_v)

                # numerator
                numerator = 0.
                for n, price in enumerate(col_c.tolist()):
                    if price > col_c[n - 1]:
                        numerator += price * col_v[n]
                    else:
                        numerator -= price * col_v[n]

                mfvs.append(numerator / denominator)
            out[:] = mfvs  
            
           
    def Net_Income_Margin():
        return or_.net_margin.latest           

    def Operating_Cashflows_To_Assets():
        return (cfs.operating_cash_flow.latest * 4.) / \
            bs.total_assets.latest

    def Price_Momentum_3M():
        return Returns(window_length=63)
    
    class Price_Oscillator(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 252

        def compute(self, today, assets, out, close):
            four_week_period = close[-20:]
            out[:] = (np.nanmean(four_week_period, axis=0) /
                      np.nanmean(close, axis=0)) - 1.
    
    def Returns_39W():
        return Returns(window_length=215)
    
    class Trendline(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 252

        # using MLE for speed
        def compute(self, today, assets, out, close):

            # prepare X matrix (x_is - x_bar)
            X = range(self.window_length)
            X_bar = np.nanmean(X)
            X_vector = X - X_bar
            X_matrix = np.tile(X_vector, (len(close.T), 1)).T

            # prepare Y matrix (y_is - y_bar)
            Y_bar = np.nanmean(close, axis=0)
            Y_bars = np.tile(Y_bar, (self.window_length, 1))
            Y_matrix = close - Y_bars

            # prepare variance of X
            X_var = np.nanvar(X)

            # multiply X matrix an Y matrix and sum (dot product)
            # then divide by variance of X
            # this gives the MLE of Beta
            out[:] = (np.sum((X_matrix * Y_matrix), axis=0) / X_var) / \
                (self.window_length)
        
    class Vol_3M(CustomFactor):
        inputs = [Returns(window_length=2)]
        window_length = 63

        def compute(self, today, assets, out, rets):
            out[:] = np.nanstd(rets, axis=0)
            
    def Working_Capital_To_Assets():
        return bs.working_capital.latest / bs.total_assets.latest
        
    all_factors = {
        'Asset Growth 3M': Asset_Growth_3M,
        'Asset to Equity Ratio': Asset_To_Equity_Ratio,
        'Capex to Cashflows': Capex_To_Cashflows,
        'EBIT to Assets': EBIT_To_Assets,
        'EBITDA Yield': EBITDA_Yield,        
        'Earnings Quality': Earnings_Quality,
        'MACD Signal Line': MACD_Signal_10d,
        'Mean Reversion 1M': Mean_Reversion_1M,
        'Moneyflow Volume 5D': Moneyflow_Volume_5d,
        'Net Income Margin': Net_Income_Margin,        
        'Operating Cashflows to Assets': Operating_Cashflows_To_Assets,
        'Price Momentum 3M': Price_Momentum_3M,
        'Price Oscillator': Price_Oscillator,
        'Return on Invest Capital': Return_On_Total_Invest_Capital,
        '39 Week Returns': Returns_39W,
        'Trendline': Trendline,
        'Vol 3M': Vol_3M,
        'Working Capital to Assets': Working_Capital_To_Assets,        
    }        
    
    return all_factors

In [3]:
n_fwd_days = 5

In [4]:
def make_pipeline():
    universe = Q1500US()
    factors = make_factors()
    factors_rank ={name: fun().rank(mask = universe) for name,fun in factors.iteritems()}
    factors_rank['Return'] = Returns(inputs = [USEquityPricing.open],mask = universe, window_length = n_fwd_days)
    pipe = Pipeline(columns = factors_rank,screen = universe)
    return pipe

In [5]:
start_timer = time()
start = pd.Timestamp("2016-03-06")
end = pd.Timestamp("2016-09-14")
factors_hist = run_pipeline(make_pipeline(),start,end)
end_time = time()

/usr/local/lib/python2.7/dist-packages/numpy/lib/nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
/usr/local/lib/python2.7/dist-packages/numpy/lib/nanfunctions.py:1202: RuntimeWarning: Degrees of freedom <= 0 for slice.
  warnings.warn("Degrees of freedom <= 0 for slice.", RuntimeWarning)


In [6]:
print('Time to run pipeline is %.2f secs' % (end_time - start_timer))

Time to run pipeline is 64.94 secs


In [7]:
factors_hist.head(10)

39 Week Returns  \
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])              289.0   
                          Equity(24 [AAPL])             489.0   
                          Equity(53 [ABMD])            1403.0   
                          Equity(62 [ABT])              548.0   
                          Equity(67 [ADSK])             800.0   
                          Equity(76 [TAP])             1380.0   
                          Equity(114 [ADBE])           1217.0   
                          Equity(122 [ADI])             717.0   
                          Equity(128 [ADM])             442.0   
                          Equity(161 [AEP])            1211.0   

                                              Asset Growth 3M  \
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])              572.0   
                          Equity(24 [AAPL])            1040.0   
                          Equity(53 [ABMD])            1313.0   
                          Equity(62 [ABT])              436.0   
                          Equity(67 [ADSK])             595.0   
                          Equity(76 [TAP])              262.0   
                          Equity(114 [ADBE])            596.0   
                          Equity(122 [ADI])            1382.0   
                          Equity(128 [ADM])             369.0   
                          Equity(161 [AEP])            1038.0   

                                              Asset to Equity Ratio  \
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])                    927.0   
                          Equity(24 [AAPL])                   619.0   
                          Equity(53 [ABMD])                    93.0   
                          Equity(62 [ABT])                    467.0   
                          Equity(67 [ADSK])                   918.0   
                          Equity(76 [TAP])                    345.0   
                          Equity(114 [ADBE])                  312.0   
                          Equity(122 [ADI])                   239.0   
                          Equity(128 [ADM])                   599.0   
                          Equity(161 [AEP])                  1024.0   

                                              Capex to Cashflows  \
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])                 243.0   
                          Equity(24 [AAPL])                691.0   
                          Equity(53 [ABMD])                627.0   
                          Equity(62 [ABT])                 410.0   
                          Equity(67 [ADSK])                681.0   
                          Equity(76 [TAP])                 431.0   
                          Equity(114 [ADBE])               697.0   
                          Equity(122 [ADI])                799.0   
                          Equity(128 [ADM])                544.0   
                          Equity(161 [AEP])               1379.0   

                                              EBIT to Assets  EBITDA Yield  \
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])             172.0          23.0   
                          Equity(24 [AAPL])           1373.0        1489.0   
                          Equity(53 [ABMD])           1222.0         236.0   
                          Equity(62 [ABT])             837.0        1416.0   
                          Equity(67 [ADSK])            248.0         269.0   
                          Equity(76 [TAP])             395.0         579.0   
                          Equity(114 [ADBE])           953.0         977.0   
                          Equity(122 [ADI])            930.0         956.0   
                          Equity(128 [ADM])            791.0        1399.0   
                          Equity(161 [AEP])            450.0        1321.0   

                                              Earnings Quality  \
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])                 NaN   
                          Equity(24 [AAPL])             1453.0   
                          

This is an important milestone. We have our ranked factor values on each day for each stock. Ranking is not absolutely necessary but has several benefits:

* it increases robustness to outliers,
* we mostly care about the relative ordering rather than the absolute values.

Also note the `Returns` column. These are the values we want to predict given the factor ranks.

Next, we are doing some additional transformations to our data:

1. Shift factor ranks to align with future returns `n_fwd_days` days in the future.
2. Find the top and bottom 30 percentile stocks by their returns. Essentially, we only care about relative movement of stocks. If we later short stocks that go down and long stocks going up relative to each other, it doesn't matter if e.g. all stocks are going down in absolute terms. Moverover, we are ignoring stocks that did not move that much (i.e. 30th to 70th percentile) to only train the classifier on those that provided strong signal. 
3. We also binarize the returns by their percentile to turn our ML problem into a classification one.

`shift_mask_data()` is a utility function that does all of these.

In [8]:
def shift_mask_data(X,Y,n_fwd_days=1,min_percent=0.2,max_percent=0.8):
    
    shifted_X = np.roll(X,n_fwd_days+1,axis = 0)

    X = shifted_X[n_fwd_days+1:]
    Y = Y[n_fwd_days+1:]
    
    n_time,n_stocks,n_factors = X.shape
    
    upper_quantile = np.nanpercentile(Y,max_percent,axis = 1)[:,np.newaxis]
    lower_quantile = np.nanpercentile(Y,min_percent,axis = 1)[:,np.newaxis]
    
    upper = (Y>upper_quantile)
    lower = (Y<lower_quantile)
    
    mask = upper | lower
    mask = mask.flatten()
    
    Y_binary = np.zeros(n_time * n_stocks)
    Y_binary[upper.flatten()] = 1
    Y_binary[lower.flatten()] = -1
    X.reshape((n_time*n_stocks,n_factors))
    
    return X[mask],Y_binary[mask]

In [9]:
results_wo_returns = factors_hist.copy()
returns = results_wo_returns.pop('Return')
Y = returns.unstack().values
X = results_wo_returns.to_panel() 
X = X.swapaxes(2, 0).swapaxes(0, 1).values # (factors, time, stocks) -> (time, stocks, factors)


In [10]:
results_wo_returns.index = results_wo_returns.index.set_levels(results_wo_returns.index.levels[1].map(lambda x: x.symbol), 1)
results_wo_returns.index = results_wo_returns.index.set_levels(results_wo_returns.index.levels[0].map(lambda x: x.date),0)
results_wo_returns.index.names = ['date','security']


In [11]:
train_size_perc = 0.8
n_time, n_stocks, n_factors = X.shape
train_size = np.int16(np.round(train_size_perc * n_time))
X_train, Y_train = X[:train_size, ...], Y[:train_size]
X_test, Y_test = X[(train_size+n_fwd_days):, ...], Y[(train_size+n_fwd_days):]



In [12]:
X_train_shift,Y_train_shift = shift_mask_data(X_train,Y_train,5)
X_test_shift, Y_test_shift = shift_mask_data(X_test, Y_test, n_fwd_days=n_fwd_days, 
                                             min_percent=50, 
                                             max_percent=50)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 101 but corresponding boolean dimension is 163115


IndexError: index 101 is out of bounds for axis 0 with size 101

In [ ]:
start_timer = time()

# Train classifier
imputer = preprocessing.Imputer()
scaler = preprocessing.MinMaxScaler()
clf = ensemble.AdaBoostClassifier(n_estimators=150) # n_estimators controls how many weak classifiers are fi

X_train_trans = imputer.fit_transform(X_train_shift)
X_train_trans = scaler.fit_transform(X_train_trans)
clf.fit(X_train_trans, Y_train_shift)

end_timer = time()

In [ ]:
print "Time to train: %0.2f secs" % (end_timer - start_timer)

In [ ]:
Y_pred = clf.predict(X_train_trans)
print('Accuracy on train set = {:.2f}%'.format(metrics.accuracy_score(Y_train_shift, Y_pred) * 100))

In [42]:
X_test_trans = imputer.transform(X_test_shift)
X_test_trans = scaler.transform(X_test_trans)
Y_pred = clf.predict(X_test_trans)
Y_pred_prob = clf.predict_proba(X_test_trans)

In [ ]:
print 'Predictions:', Y_pred
print 'Probabilities of class == 1:', Y_pred_prob[:, 1] * 100

In [ ]:
print('Accuracy on test set = {:.2f}%'.format(metrics.accuracy_score(Y_test_shift, Y_pred) * 100))
print('Log-loss = {:.5f}'.format(metrics.log_loss(Y_test_shift, Y_pred_prob)))

In [ ]:
feature_importances = pd.Series(clf.feature_importances_, index=results_wo_returns.columns)
feature_importances.sort(ascending=False)
ax = feature_importances.plot(kind='bar')
ax.set(ylabel='Importance (Gini Coefficient)', title='Feature importances');